# MST-algorithm

Firstly we use classical clusterization algorythm using Minimum Spanning Tree, counting distances between the particles. 

#### Dependencies:

In [ ]:
import csv
import pandas as pd
import numpy as np
import networkx as nx


#### Parsing data, writing into pandas dataset

In [ ]:
def parse_coordinates_file(file_path):
    coordinates = []

    with open(file_path, "r") as f:
        lines = f.readlines()

    for line in reversed(lines):
        line = line.strip()
        if not line.startswith("0.") and line:
            break
        row = [line.split()[i] for i in range(1, 4)]
        coordinates.append([float(x) for x in row])

    coordinates_df = pd.DataFrame(coordinates, columns=["rx", "ry", "rz"])

    print(f"Parsed {len(coordinates_df)} coordinates from file")
    return coordinates_df

# Here is the example usage, I avoid loading big file with data on GitHub
# file_path = "bicuskyrm.f14"
# coordinates_df = parse_coordinates_file(file_path)

# coordinates_df.head()

Parsed 375 coordinates from file


,rx,ry,rz
0,66.139954,-35.237257,62.625780
1,44.576542,21.132161,122.363250
2,17.329893,62.270253,123.846110
3,27.731119,-85.652354,-15.954348
4,46.628560,42.162045,121.780590


In [48]:
coordinates_df.to_csv("coordinates.csv", index=False)

#### Building weighted graph on our points

In [50]:
def build_weighted_graph(coords_df, sample_size=None):
    if sample_size and sample_size < len(coords_df):
        df_sample = coords_df.sample(sample_size, random_state=42)
    else:
        df_sample = coords_df

    points = df_sample[["rx", "ry", "rz"]].values

    G = nx.Graph()

    for i, (_, row) in enumerate(df_sample.iterrows()):
        G.add_node(i, pos=(row["rx"], row["ry"], row["rz"]))

    # Calculate distances and add edges without using tqdm
    for i in range(len(df_sample)):
        for j in range(i + 1, len(df_sample)):
            dist = np.sqrt(((points[i] - points[j]) ** 2).sum())
            G.add_edge(i, j, weight=dist)

    print(
        f"Graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges"
    )
    return G


graph = build_weighted_graph(coordinates_df)


Graph built with 375 nodes and 70125 edges


In [51]:
mst = nx.minimum_spanning_tree(graph, algorithm='prim')

print(f"MST created with {mst.number_of_nodes()} nodes and {mst.number_of_edges()} edges")

edge_weights = [data['weight'] for u, v, data in mst.edges(data=True)]
total_weight = sum(edge_weights)
avg_weight = np.mean(edge_weights)
max_weight = np.max(edge_weights)
min_weight = np.min(edge_weights)

print(f"Total MST weight: {total_weight:.2f}")
print(f"Average edge weight: {avg_weight:.2f}")
print(f"Maximum edge weight: {max_weight:.2f}")
print(f"Minimum edge weight: {min_weight:.2f}")


MST created with 375 nodes and 374 edges
Total MST weight: 6561.10
Average edge weight: 17.54
Maximum edge weight: 51.31
Minimum edge weight: 0.42


In [52]:
filtered_mst = nx.Graph()

for node in mst.nodes():
    filtered_mst.add_node(node)
    if "pos" in mst.nodes[node]:
        filtered_mst.nodes[node]["pos"] = mst.nodes[node]["pos"]

filtered_edges = [
    (u, v, data) for u, v, data in mst.edges(data=True) if data["weight"] <= 3.0
]
filtered_mst.add_edges_from([(u, v, data) for u, v, data in filtered_edges])

print(
    f"Original MST had {mst.number_of_nodes()} nodes and {mst.number_of_edges()} edges"
)
print(
    f"Filtered MST has {filtered_mst.number_of_nodes()} nodes and {filtered_mst.number_of_edges()} edges"
)

connected_components = list(nx.connected_components(filtered_mst))
print(f"Number of connected components after filtering: {len(connected_components)}")
print(f"Sizes of connected components: {[len(comp) for comp in connected_components]}")


filtered_weights = [data["weight"] for u, v, data in filtered_mst.edges(data=True)]
print(f"Average edge weight in filtered MST: {np.mean(filtered_weights):.2f}")
print(f"Maximum edge weight in filtered MST: {np.max(filtered_weights):.2f}")
print(f"Minimum edge weight in filtered MST: {np.min(filtered_weights):.2f}")


Original MST had 375 nodes and 374 edges
Filtered MST has 375 nodes and 55 edges
Number of connected components after filtering: 320
Sizes of connected components: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 4, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 4, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 2, 1, 1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, 3, 1, 2, 4, 1, 1, 2, 1, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [53]:
connected_components = list(nx.connected_components(filtered_mst))

connected_components.sort(key=len, reverse=True)

max_component_size = max(len(comp) for comp in connected_components)
csv_data = []

for i in range(max_component_size):
  row = []
  for comp in connected_components:
    comp_list = list(comp)
    if i < len(comp_list):
      row.append(comp_list[i])
    else:
      row.append("")
  csv_data.append(row)

with open('mst_clusters.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  
  headers = [f"Component_{i+1}" for i in range(len(connected_components))]
  writer.writerow(headers)
  
  writer.writerows(csv_data)

print(f"Saved {len(connected_components)} clusters to mst_clusters.csv")
print(f"Largest cluster has {max_component_size} nodes")

clusters = pd.read_csv('mst_clusters.csv')
clusters

Saved 320 clusters to mst_clusters.csv
Largest cluster has 7 nodes


,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,Component_9,Component_10,...,Component_311,Component_312,Component_313,Component_314,Component_315,Component_316,Component_317,Component_318,Component_319,Component_320
0,355,356.0,208.0,130.0,177.0,185.0,162.0,168.0,228.0,353.0,...,361.0,363.0,364.0,365.0,367.0,368.0,371.0,372.0,373.0,374.0
1,332,165.0,315.0,242.0,346.0,202.0,148.0,362.0,277.0,323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,206,340.0,238.0,205.0,225.0,250.0,303.0,317.0,182.0,190.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,370,285.0,119.0,366.0,198.0,290.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,211,158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
